In [1]:
import time
import _pickle as cPickle
import os

In [2]:
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as Data


In [4]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.device(0)

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.get_device_name(0)

'TITAN Xp'

In [8]:
# Device configuration
device = torch.device('cuda:0')
print('Using device:', device)

Using device: cuda:0


In [9]:
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
# torch.cuda.set_device(1)

7104
True


In [10]:
torch.backends.cudnn.enabled = True

In [11]:
# Hyper-parameters 
input_size = 47
hidden_size = 64
num_classes = 1
num_epochs = 10000
batch_size = 8
learning_rate = 0.002


# Data loader

In [12]:
class DatasetPeak(Dataset):
    def __init__(self, data):
        #     index: ith sensor

        x = data.iloc[:, 1:].values.astype(np.float64)
        self.x = torch.from_numpy(x)

        label = data.iloc[:, 0]
        arr = np.array([label])

        self.label = torch.from_numpy(arr).view(-1, 1)
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        label = self.label[index]
        return x, label

In [13]:
p3c1_idx = []
filename = []
in_z0 = round(-287*2)
out_c4 = round(93*2)
out_board = round(130*2)
temp_area_lsit = ['z0','z1','z2','z3','z4','z5','z6','z7','z8',
                  'p1','p2','p3',
                  'c1','c2','c3','c4',
                  ]
different_area_list = ['in','constant','warming','cooling']
conveyor_ordinal = [1,2,3,0]
z0_stay_time = 30
other_stay_time = 23.5
sensor_column = ['Y.'+str(i) for i in range(20)]

# path = "Z:\2. Projects\英業達\資料\2018.09資料\桃園SMT爐溫監測\reflowdata\train"
with open('train_peak.pkl', 'rb') as f:

# with open('train_peak.pkl', 'rb') as f:
    train_peak = cPickle.load(f)  
    
    
with open('test_peak.pkl', 'rb') as f:
    test_peak = cPickle.load(f)  

In [14]:
train_peak[0].shape

(56, 48)

In [15]:
# index = 0
# train_x, train_y = train_peak[index].iloc[:, 1:], train_peak[index].iloc[:, 0]

In [16]:
def load_dataset(index):
    train_dataset = DatasetPeak(train_peak[index])
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
#     train_iter = iter(train_loader)
#     x, y = next(train_iter)
#     print(x.shape)
#     print(y.shape)
    return train_loader

In [17]:
def load_test_dataset(index):
    test_dataset = DatasetPeak(test_peak[index])
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
#     test_iter = iter(test_loader)
    return test_loader

In [16]:
# index = 0
# train_dataset = DatasetPeak(train_peak[index])
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# train_iter = iter(train_loader)
# print(type(train_iter))

# Model setting

In [18]:
# mean diff = 1.0985968112945557
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc2 = nn.Linear(128, 64)  
        self.fc3 = nn.Linear(64, 32)  
        self.fc4 = nn.Linear(32, 1)  
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

In [24]:
# mean diff = 
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
        ) 
        self.fc2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
        ) 
        self.fc3 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.25),
        ) 
        self.fc4 = nn.Sequential(
            nn.Linear(32, 1),
        ) 

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

In [25]:
input_size = 20
model = NeuralNet(input_size).to(device)
model = model.cuda(device)

In [26]:
# Loss and optimizer
criterion = nn.MSELoss()# Mean Squared Loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [27]:
index = 0
# for index in range(20):
train_loader = load_dataset(index)
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (x, labels) in enumerate(train_loader):  
        t = time.time()
        # Move tensors to the configured device
        x = x.cuda(device).float()

        labels = labels.cuda(device).float()

        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("cost time =", time.time()-t)
    if (epoch+1) % 100 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, loss.item()))

RuntimeError: size mismatch, m1: [16 x 47], m2: [20 x 128] at c:\programdata\miniconda3\conda-bld\pytorch_1533096106539\work\aten\src\thc\generic/THCTensorMathBlas.cu:249

In [22]:
import pprint

In [23]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
output = []
index = 0
with torch.no_grad():
    correct = 0
    total = 0
    test_loader = load_test_dataset(index)
    for x, real_y in test_loader:
        x = x.cuda(device).float()
        real_y = real_y.cuda(device).float()
        test_y = model(x)
        real_y, test_y = real_y[0], test_y[0]
#         _, predicted = torch.max(outputs.data, 1)
        output.append(pd.DataFrame({'sensor':index,'origin':real_y,'predict':test_y,'diff':abs(real_y-test_y)}))
output_all = pd.concat(output)
print("mean diff =", output_all["diff"].mean())
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output_all)


#     print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    

RuntimeError: size mismatch, m1: [1 x 47], m2: [20 x 128] at c:\programdata\miniconda3\conda-bld\pytorch_1533096106539\work\aten\src\thc\generic/THCTensorMathBlas.cu:249

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')